In [1]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

In [ ]:
#import org.apache.spark.sql.types._

In [190]:
#jsonpath = "abfss://csiwkproot@csiwkpstore.dfs.core.windows.net/vialtest/0_c63a2568cd1a4d99922c68f3b6283d3f_1.json"
jsonpath = "abfss://csiwkproot@csiwkpstore.dfs.core.windows.net/vialtest/*.json"

In [191]:
from pyspark.sql.types import *

schema = StructType([
  StructField("ConnectionDeviceId", StringType()),
  StructField("EnqueuedTime", StringType()),
  StructField("EventEnqueuedUtcTime", StringType()),
  StructField("EventProcessedUtcTime", StringType()),
  StructField("gatewayData", ArrayType(
      StructType([
          StructField("mimeType", StringType()),
          StructField("model_id", StringType()),
          StructField("tag_id", StringType()),
          StructField("vqts", ArrayType(
              StructType([
                      StructField('q', StringType()),
                      StructField('t', StringType()),
                      StructField('v', StringType())
                  ])
                )
            )
        ])
    )
)
])

In [192]:
df = spark.read.schema(schema).json(jsonpath)

In [ ]:
#display(df)

In [193]:
df.printSchema

In [ ]:
#display(df.select("ConnectionDeviceId", "gatewayData.tag_id", "gatewayData.vqts"))

In [194]:
from pyspark.sql.functions import split, explode

#df.select(col("ConnectionDeviceId"), explode(col("gatewayData.tag_id")).alias("tag_id")).show()

In [195]:
import pyspark.sql.functions as f
from pyspark.sql.types import *

df = df.withColumn('tag_id', f.explode('gatewayData.tag_id'))
df = df.withColumn('vqts', f.explode('gatewayData.vqts'))

In [ ]:
display(df)

In [196]:
df = df.withColumn('vqts1', f.explode('vqts'))

In [ ]:
display(df)

In [ ]:
from pyspark.sql.functions import *

df = df.withColumn('q', regexp_replace(df["vqts.q"], '[', ''))
df = df.withColumn('q', regexp_replace(df["vqts.q"], ']', ''))
df = df.withColumn('t', regexp_replace(df["vqts.t"], '[', ''))
df = df.withColumn('t', regexp_replace(df["vqts.t"], ']', ''))
df = df.withColumn('v', regexp_replace(df["vqts.v"], '[', ''))
df = df.withColumn('v', regexp_replace(df["vqts.v"], ']', ''))

In [ ]:
df = df.withColumn('q1', regexp_replace(df["q"], '[', ''))
df = df.withColumn('t1', regexp_replace(df["t"], '[', ''))
df = df.withColumn('v1', regexp_replace(df["v"], '[', ''))

In [ ]:
#display(df)

In [186]:
#df.select("ConnectionDeviceId", "tag_id", "vqts1.q", "vqts1.t", "vqts1.v").show()

In [198]:
display(df.select("ConnectionDeviceId", "tag_id", "vqts1.q", "vqts1.t", "vqts1.v"))

In [188]:
df.select("ConnectionDeviceId", "tag_id", "vqts1.q", "vqts1.t", "vqts1.v").repartition(1).write.mode("overwrite").parquet('abfss://csiwkproot@csiwkpstore.dfs.core.windows.net/rawdata/')

In [199]:
df1 = df.select("ConnectionDeviceId", "tag_id", "vqts1.q", "vqts1.t", "vqts1.v")

In [204]:
from pyspark.sql.functions import *

pivotDF = df1.groupBy('t', 'tag_id').pivot("tag_id").agg(first("v"))
pivotDF.show()